this nb is responsible for scraping the data and handling all the messy html stuff. gives you a csv file containing the pq paragraphs and a bit of metadata.

sometimes code is too fast for browser driver. in such cases, try increasing sleep time.

In [1]:
from bs4 import BeautifulSoup as bs
from datetime import datetime
import math
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time 
import os
from enum import Enum
import re
import pandas as pd
 

In [2]:
class ReportSection(Enum):
    WRITTEN = 'Written Answers to Questions'
    WRITTEN_NA = 'Written Answers to Questions for Oral Answer Not Answered by End of Question Time'
    ORAL = 'Oral Answers to Questions'

In [3]:
DRIVER_PATH = 'chromedriver_win32/chromedriver.exe'
SEARCH_TERM = 'asked the' # can be blank
# PARLIAMENT_INDEX = -1 # legislative assembly = 1, 1st parliament = 2, etc etc. -1 to ignore this field.

In [4]:
def element_completely_viewable(driver, elem):
    elem_left_bound = elem.location.get('x')
    elem_top_bound = elem.location.get('y')
    elem_width = elem.size.get('width')
    elem_height = elem.size.get('height')
    elem_right_bound = elem_left_bound + elem_width
    elem_lower_bound = elem_top_bound + elem_height

    win_upper_bound = driver.execute_script('return window.pageYOffset')
    win_left_bound = driver.execute_script('return window.pageXOffset')
    win_width = driver.execute_script('return document.documentElement.clientWidth')
    win_height = driver.execute_script('return document.documentElement.clientHeight')
    win_right_bound = win_left_bound + win_width
    win_lower_bound = win_upper_bound + win_height

    return all((win_left_bound <= elem_left_bound,
                win_right_bound >= elem_right_bound,
                win_upper_bound <= elem_top_bound,
                win_lower_bound >= elem_lower_bound)
              )

def find_element(driver, elem):
    y_height = 0
    while not element_completely_viewable(driver, elem):
        driver.execute_script(f"window.scrollTo(0, {y_height})") 
        y_height += 100
        time.sleep(0.5)

def find_and_click(driver, elem):
    find_element(driver, elem)
    elem.click()



In [5]:
driver = webdriver.Chrome(DRIVER_PATH) 
 
page_url = 'https://sprs.parl.gov.sg/search/home' 
driver.get(page_url) 
driver.maximize_window() 
driver.implicitly_wait(20) 
time.sleep(2) 
 
# Get search box and fill it up 

search = driver.find_element(by=By.XPATH, value='/html/body/app-root/app-search/div/div[2]/div[2]/div[1]/div/div[1]/input')
search.send_keys(SEARCH_TERM) 

# exact phrase search

search_opt = driver.find_element(by=By.XPATH, value='/html/body/app-root/app-search/div/div[2]/div[2]/div[2]/div/div[1]/select')
find_and_click(driver, search_opt)
exact_search_opt = driver.find_element(by=By.XPATH, value='//*[@id="divmpscreen2"]/div[2]/div[2]/div/div[1]/select/option[3]')
find_and_click(driver, exact_search_opt)
 
# Uncomment following two lines to only search in titles # 
#checkbox = driver.find_element_by_xpath('//*[@id="divmpscreen2"]/div[2]/div[1]/div/label/input')
#checkbox.click() 

# from 1st parliament to 14th. ignore legis. assembly cuz the parl website has no list of members.
for i in range(2, 16):
    session = driver.find_element(
        by=By.XPATH, 
        value=f'/html/body/app-root/app-search/div/div[2]/div[2]/div[1]/div/div[3]/select/option[{i}]')
    session.click() 
 
# advanced search - only look for oral and written answers to questions

adv_search_opt = driver.find_element(by=By.XPATH, value='/html/body/app-root/app-search/div/div[3]/div/div[1]/h5/a')
find_and_click(driver, adv_search_opt)
section_selection = driver.find_element(by=By.XPATH, value='/html/body/app-root/app-search/div/div[3]/div/div[2]/div/div/div/div/div[3]/div/select')
find_element(driver, section_selection)
for selection_option_index in [13, 19, 20]: # corresponds to answers to questions    
    option = driver.find_element(
        by=By.XPATH, 
        value=f'/html/body/app-root/app-search/div/div[3]/div/div[2]/div/div/div/div/div[3]/div/select/option[{selection_option_index}]')
    option.click()


# Find submit element and press enter

submit = driver.find_element(by=By.XPATH, value='/html/body/app-root/app-search/div/div[2]/div[2]/div[3]/div/button[2]') 
submit.send_keys(Keys.RETURN) 

print('Search parameters submitted.') 

# Switch window and check for number of search results 

driver.switch_to.window(driver.window_handles[1])
driver.implicitly_wait(20) 
time.sleep(2) 

try:
    num_res_label = driver.find_element(by=By.CLASS_NAME, value='showingResults')
except: 
    print('Error when finding num_res_label')
    
num_res_text = num_res_label.get_attribute('innerHTML')
    
if 'No' in num_res_text:
    print('no results found') # but site is quite erratic, may take a few tries to get results (idk why)
else:
    num_results = int(num_res_text[num_res_text.rfind(' ')+1:])
    num_pages = math.ceil(num_results / 20)
    print(f'{num_results} results found spanning {num_pages} page(s)')

C:\Users\thisi\AppData\Local\Temp\ipykernel_21904\1286528577.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(DRIVER_PATH)


Search parameters submitted.
11904 results found spanning 596 page(s)


In [ ]:
docs_saved = 0
pages_seen = 0

pq_paras = []

while (True):
    driver.switch_to.window(driver.window_handles[1])                  

    elems = driver.find_elements(by=By.XPATH, value='/html/body/app-root/app-result/div/div/div[2]/div/div/div/div/div/div[1]/table/tbody')
    links = list(map(lambda elem: elem.find_elements(by=By.XPATH, value='.//tr[1]/td[2]/a')[0], elems))
    
    for link in links:
        find_and_click(driver, link)
        
        # Switch to page with content and get URL name
        time.sleep(1)
        driver.switch_to.window(driver.window_handles[2])         
        item_key = driver.current_url.split('/')[-1]         
        item_key = item_key.replace('?', '_') # Replace ? because it would be an invalid filename 

        file = item_key
        page_source = driver.page_source
        
        if file[0:4] == 'sprs':
            is_sprs = True
        else:
            is_sprs = False
            
        soup = bs(page_source, 'html.parser')

        if is_sprs:
            # extract metadata from the table at the top of the page
            topic_table = soup.find('table', {'class': 'topic'})
            topic_text = topic_table.get_text()

            parl_no_label_span = re.match(r'Parliament No:', topic_text).span()
            topic_text = topic_text[parl_no_label_span[1]:]
            parl_no = int(re.match(r'\d+', topic_text).group())

            sit_date_label_span = re.match(r'.*Sitting Date:', topic_text).span()
            topic_text = topic_text[sit_date_label_span[1]:]
            sit_date = datetime.strptime(re.match(r'\d+-\d+-\d+', topic_text).group(), '%d-%m-%Y')

            # extract report section via filename
            if 'oral-answer' in file:
                section = ReportSection.ORAL
            elif 'written-answer-na' in file:
                section = ReportSection.WRITTEN_NA
            elif 'written-answer' in file:
                section = ReportSection.WRITTEN
            else:
                raise 'unidentified section (sprs)'
            section = section.value

            # extract paragraphs from report in the form of rows of table
            report_table = soup.find('div', {'class': 'reportTable'})
            report_table_rows = report_table.findChildren()

            # find out which rows correspond to pqs and which rows don't.
            for row in report_table_rows:
                paragraph = row.get_text()
                paragraph = paragraph.replace('\xa0', ' ')
                if not re.match(r'\d+.{8,75} asked the .+', paragraph):
                    continue
                paragraph_no_num = paragraph[
                    re.search('\d+', paragraph).span()[1]:]
                pq_paras.append((paragraph_no_num.strip(), sit_date, parl_no, section))
        else:
            # extract metadata from meta elements directly
            sit_date_str = soup.find('meta', {'name': 'Sit_Date'})['content']
            parl_no_str = soup.find('meta', {'name': 'Parl_No'})['content']
            section_str = soup.find('meta', {'name': 'Sect_Name'})['content']

            sit_date = datetime.strptime(sit_date_str, '%Y-%m-%d')
            parl_no = int(parl_no_str)
            if 'NOT REACHED' in section_str:
                section = ReportSection.WRITTEN_NA
            elif 'ORAL' in section_str:
                section = ReportSection.ORAL
            elif 'WRITTEN' in section_str:
                section = ReportSection.WRITTEN
            else:
                raise 'unidentified section (non sprs)'
            section = section.value

            # extract paragraphs from table in the form of raw text (since each paragraph doesn't have its own element)
            report_table = soup.find('div', {'class': 'reportTable'})
            raw_text = report_table.get_text()

            # look for things like 1. 2. etc
            first_number_dot_occurrence = re.search(r'\d+\. .{8,75} asked the ', raw_text)

            if not first_number_dot_occurrence:
                continue

            # cut off everything before the actual body of the text
            raw_text = raw_text[first_number_dot_occurrence.span()[0]:]

            # formatting stuff
            raw_text = raw_text.replace('\n\n', ' ')
            raw_paras = raw_text.split('\xa0\xa0\xa0\xa0')

            new_pq_paras = list(
                map(lambda s: (s[re.search('\d+\.', s).span()[1]:].strip(), sit_date, parl_no, section), # remove index, add sitting date, parl no, section
                    map(lambda s: re.sub('Column: \d+', '', s.replace('  ', ' ').strip()), # remove column indicators and extra spaces
                        filter(lambda para: re.match(r'\d+\. .{8,75} asked the ', para), raw_paras)))) # keep only paragraphs that are pqs.

            pq_paras += new_pq_paras

        docs_saved += 1
        if docs_saved % 20 == 0:
            print(f'Saved {docs_saved} / {num_results} docs at {datetime.now().strftime("%H%M")}')

        # Close tab         
        driver.close()
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(1)
    
    if docs_saved == num_results:
        break
    elif docs_saved == 20: # if we're on the first results page then there is no previous page button
        next_page = driver.find_element(
            by=By.XPATH, 
            value='/html/body/app-root/app-result/div/div/div[2]/div/div/div/div/div/div[1]/div[3]/section/ul/li[1]/a')
    else: # if we're not on the first results page then there is a previous page button
        next_page = driver.find_element(
            by=By.XPATH, 
            value='/html/body/app-root/app-result/div/div/div[2]/div/div/div/div/div/div[1]/div[3]/section/ul/li[3]/a')
    
    find_and_click(driver, next_page)
    pages_seen += 1
    print(f'Seen {pages_seen} / {num_pages} pages')

    # Sleep momentarily because next page takes a while to load     
    time.sleep(2) 


Saved 20 / 11904 docs at 1520
Seen 1 / 596 pages
Saved 40 / 11904 docs at 1522
Seen 2 / 596 pages
Saved 60 / 11904 docs at 1524
Seen 3 / 596 pages
Saved 80 / 11904 docs at 1526
Seen 4 / 596 pages
Saved 100 / 11904 docs at 1528
Seen 5 / 596 pages
Saved 120 / 11904 docs at 1530
Seen 6 / 596 pages
Saved 140 / 11904 docs at 1532
Seen 7 / 596 pages
Saved 160 / 11904 docs at 1533
Seen 8 / 596 pages


In [ ]:
pq_paras_df = pd.DataFrame(pq_paras, columns=['para', 'sit_date', 'parl_no', 'section'])
pq_paras_df.to_csv('pq_paras.csv', index=False)